## Supervised Learning using BERT for Sentiment Analysis and Notation Prediction

### Data Extraction and Preprocessing (we follow the same process as in data_cleaning but we keep a different sample of the data)

In [4]:
import pandas as pd
import nltk
from cleantext import clean
import tqdm
import numpy as np
from deep_translator import GoogleTranslator
import torch

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [22]:
torch.cuda.is_available()

True

In [23]:
reviews_df = pd.read_csv("data/reviews.csv", sep=";")
reviews_df.head(3)

,category_link,company_link,company_name,score,description,review_page_nb,review_link,review_score,review_title,review_text,category
0,https://fr.trustpilot.com/categories/food_beverages_tobacco?verified=true,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9","Le Fourgon vous livre vos boissons consignées à domicile :\n🍶 la commande se passe sur lefourgon.com : bières, jus, sodas, eaux, lait, vins, soupes, spririteux, & co.\n🚚 nous vous livrons gratuitement chez vous sur le créneau choisi\n🌱 et au passage suivant on récupère vos bouteilles vides qu’on renvoie lavées au producteur pour ré-emploi #zerodechet",329,https://fr.trustpilot.com/reviews/65a5388a60d6a4425263fe42,5,Application conviviale pour passer ses…,Application conviviale pour passer ses commandes. Manquent juste les ingrédients pas évidents à trouver.Notification par SMS peu de temps avant le créneau choisi pour un créneau précis de 20 minutes.Livreurs agréables.Merci le Fourgon !,food_beverages_tobacco
1,https://fr.trustpilot.com/categories/food_beverages_tobacco?verified=true,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9","Le Fourgon vous livre vos boissons consignées à domicile :\n🍶 la commande se passe sur lefourgon.com : bières, jus, sodas, eaux, lait, vins, soupes, spririteux, & co.\n🚚 nous vous livrons gratuitement chez vous sur le créneau choisi\n🌱 et au passage suivant on récupère vos bouteilles vides qu’on renvoie lavées au producteur pour ré-emploi #zerodechet",329,https://fr.trustpilot.com/reviews/65a53245a223c5f420a9cee3,5,Très facile pour la commande,"Très facile pour la commande, très rapide et livraison très professionnelle! Le livreur a apporté les bacs chez nous car ils étaient lourds! 12 bouteilles par bacs... soit 12kg sans compter le poids des bouteilles. Très pro et sympa!Vraiment chouette et écologique! On recommandera sans hésiter! Fini les microparticules de plastique!",food_beverages_tobacco
2,https://fr.trustpilot.com/categories/food_beverages_tobacco?verified=true,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9","Le Fourgon vous livre vos boissons consignées à domicile :\n🍶 la commande se passe sur lefourgon.com : bières, jus, sodas, eaux, lait, vins, soupes, spririteux, & co.\n🚚 nous vous livrons gratuitement chez vous sur le créneau choisi\n🌱 et au passage suivant on récupère vos bouteilles vides qu’on renvoie lavées au producteur pour ré-emploi #zerodechet",329,https://fr.trustpilot.com/reviews/659f0e15dd570fb981a1d433,5,Première expérience réussie !,"Pour nous, c'était une première.Ravis d'avoir trouvé un système de consigne pour un choix plus large de produits qu'en magasin.Créneau horaire de livraison respecté, avec une information précise sur l'approche du livreur.Livreur sympathique, souriant et serviable.À bientôt pour une nouvelle commande !",food_beverages_tobacco


In [24]:
reviews_df = reviews_df[["company_name", "score", "review_score", "review_title", "review_text", "category"]]

In [25]:
# we merge the title and the text of the review
reviews_df["review"] = reviews_df["review_title"] + " " + reviews_df["review_text"]
reviews_df.head(3)

,company_name,score,review_score,review_title,review_text,category,review
0,Le Fourgon,"4,9",5,Application conviviale pour passer ses…,Application conviviale pour passer ses commandes. Manquent juste les ingrédients pas évidents à trouver.Notification par SMS peu de temps avant le créneau choisi pour un créneau précis de 20 minutes.Livreurs agréables.Merci le Fourgon !,food_beverages_tobacco,Application conviviale pour passer ses… Application conviviale pour passer ses commandes. Manquent juste les ingrédients pas évidents à trouver.Notification par SMS peu de temps avant le créneau choisi pour un créneau précis de 20 minutes.Livreurs agréables.Merci le Fourgon !
1,Le Fourgon,"4,9",5,Très facile pour la commande,"Très facile pour la commande, très rapide et livraison très professionnelle! Le livreur a apporté les bacs chez nous car ils étaient lourds! 12 bouteilles par bacs... soit 12kg sans compter le poids des bouteilles. Très pro et sympa!Vraiment chouette et écologique! On recommandera sans hésiter! Fini les microparticules de plastique!",food_beverages_tobacco,"Très facile pour la commande Très facile pour la commande, très rapide et livraison très professionnelle! Le livreur a apporté les bacs chez nous car ils étaient lourds! 12 bouteilles par bacs... soit 12kg sans compter le poids des bouteilles. Très pro et sympa!Vraiment chouette et écologique! On recommandera sans hésiter! Fini les microparticules de plastique!"
2,Le Fourgon,"4,9",5,Première expérience réussie !,"Pour nous, c'était une première.Ravis d'avoir trouvé un système de consigne pour un choix plus large de produits qu'en magasin.Créneau horaire de livraison respecté, avec une information précise sur l'approche du livreur.Livreur sympathique, souriant et serviable.À bientôt pour une nouvelle commande !",food_beverages_tobacco,"Première expérience réussie ! Pour nous, c'était une première.Ravis d'avoir trouvé un système de consigne pour un choix plus large de produits qu'en magasin.Créneau horaire de livraison respecté, avec une information précise sur l'approche du livreur.Livreur sympathique, souriant et serviable.À bientôt pour une nouvelle commande !"


In [26]:
clean(reviews_df["review"].values[0], no_emoji=True, no_line_breaks=True, lower=False).replace("#", "")

'Application conviviale pour passer ses Application conviviale pour passer ses commandes. Manquent juste les ingredients pas evidents a trouver.Notification par SMS peu de temps avant le creneau choisi pour un creneau precis de 20 minutes.Livreurs agreables.Merci le Fourgon !'

In [27]:
reviews_df["review"] = reviews_df["review"].apply(lambda x: clean(x, no_emoji=True, no_line_breaks=True, lower=False).replace("#", ""))

In [28]:
reviews_df["review"].values[100]

"Exceptionnel La brulerie Belleville est devenue mon unique fournisseur en cafe, le cafe est vraiment d'une qualite exceptionnelle ! De plus, le service apres-vente est d'une qualite humaine devenue bien rare a present !"

#### We Keep only a sample of the data (the top 100 companies with the most reviews) Because of the traduction cost of the "review" column

In [29]:
# number of reviews per company
reviews_df["company_name"].value_counts()[:10]

company_name
Hardloop           236
Alltricks          210
Ekosport           140
vertbaudet         140
Mode Tactique      120
Weenect            120
AAAEP              120
Handball Store     111
Meyclub            100
Bleen              100
Name: count, dtype: int64

In [30]:
# number of reviews in all the dataset
len(reviews_df)

235503

In [31]:
# print the first company name
reviews_df["company_name"].value_counts().index[0]

'Hardloop\xa0'

In [32]:
reviews_df["company_name"] = reviews_df["company_name"].apply(lambda x: x.replace("\xa0", "") if x != "\xa0" else x)
reviews_df["company_name"].value_counts().index[0]

'Hardloop'

In [33]:
# we keep only the reviews of the first 100 companies in terms of number of reviews
revsample_df = reviews_df[reviews_df["company_name"].isin(reviews_df["company_name"].value_counts()[:100].index)]
revsample_df["company_name"].value_counts().sum()

8972

In [34]:
revsample_df["company_name"].value_counts()[:10]

company_name
Hardloop          236
Alltricks         210
vertbaudet        140
Ekosport          140
Weenect           120
Mode Tactique     120
AAAEP             120
Handball Store    111
Plaquedeces.fr    100
JACQUEMUS         100
Name: count, dtype: int64

In [35]:
revsample_df = revsample_df.dropna(ignore_index=True)

In [36]:
# we display all the lines of the first company
pd.set_option("display.max_colwidth", None)
revsample_df[revsample_df["company_name"] == "Hardloop"]

,company_name,score,review_score,review_title,review_text,category,review
7658,Hardloop,"4,5",1,Première commande sur ce site,"Première commande sur ce site, et extrêmement déçue !Pantalon de randonnée commandé pour un cadeau de Noël début décembre, il n’est toujours pas réceptionné malgré les dates indiquées sur le site. Diverses relances auprès du service client par e-mail et Instagram afin d’obtenir à minima un retour sur la livraison ou sur un remboursement éventuel pour l’acheter ailleurs : aucun retour.Une honte.À fuir…Merci de supprimer mon adresse e-mail privée à ce commentaire (méthode très douteuse pour que je supprime mon commentaire négatif).",sports,"Premiere commande sur ce site Premiere commande sur ce site, et extremement decue !Pantalon de randonnee commande pour un cadeau de Noel debut decembre, il n'est toujours pas receptionne malgre les dates indiquees sur le site. Diverses relances aupres du service client par e-mail et Instagram afin d'obtenir a minima un retour sur la livraison ou sur un remboursement eventuel pour l'acheter ailleurs : aucun retour.Une honte.A fuirMerci de supprimer mon adresse e-mail privee a ce commentaire (methode tres douteuse pour que je supprime mon commentaire negatif)."
7659,Hardloop,"4,5",1,Pantalon à 150€ reçu 10 jours après la…,"Pantalon à 150€ reçu 10 jours après la commande. Pour cause de taille trop grande, impossible d'obtenir un bon de retour pour retourner l'article, car le site émet systématiquement un message d'erreur quand on en fait la demande. Le numéro de tel du prétendu standard n'est en fait qu'un simple répondeur, où il n'est même pas possible de laisser un message.Finalement, après deux mails envoyés, j'ai fini par recevoir un bon de retour imprimable. Pas de remboursement reçu un mois après l'envoi... mais reçu immédiatement après après que j'aie renvoyé un mailde reclamation. À fuir.",sports,"Pantalon a 150 recu 10 jours apres la Pantalon a 150 recu 10 jours apres la commande. Pour cause de taille trop grande, impossible d'obtenir un bon de retour pour retourner l'article, car le site emet systematiquement un message d'erreur quand on en fait la demande. Le numero de tel du pretendu standard n'est en fait qu'un simple repondeur, ou il n'est meme pas possible de laisser un message.Finalement, apres deux mails envoyes, j'ai fini par recevoir un bon de retour imprimable. Pas de remboursement recu un mois apres l'envoi... mais recu immediatement apres apres que j'aie renvoye un mailde reclamation. A fuir."
7660,Hardloop,"4,5",5,Equipe très réactive en cas de problème…,Equipe très réactive en cas de problème sur une commande pour trouver des solutions.,sports,Equipe tres reactive en cas de probleme Equipe tres reactive en cas de probleme sur une commande pour trouver des solutions.
7661,Hardloop,"4,5",4,Très bon service,"Beaucoup de choix, produits bien emballés, et livraison rapide.",sports,"Tres bon service Beaucoup de choix, produits bien emballes, et livraison rapide."
7662,Hardloop,"4,5",5,Parfait,"Parfait, livraison rapide et service optimal, je recommande",sports,"Parfait Parfait, livraison rapide et service optimal, je recommande"
...,...,...,...,...,...,...,...
8676,Hardloop,"4,0",5,"Competitive price, rapid delivery","Competitive price, rapid delivery to Poland",vehicles_transportation,"Competitive price, rapid delivery Competitive price, rapid delivery to Poland"
8677,Hardloop,"4,0",1,Szczytem chamstwa nie jest dostarczyć…,"Szczytem chamstwa nie jest dostarczyć produkt, który zgodnie z prawem zakupiłem, i który jest dostępny w sklepie, tylko dlatego, że właścicielowi nie podoba się kwota, za którą został sprzedany. W systemie była widoczna cena w promocji. I z niej korzystając kupiłem kurtkę Helly Hannsena. Anulowanie mojego kupna i zwrot pieniędzy zasłaniając się rzekomym błędem w wyświetlaniu ceny jest co najmniej nie na miejscu. Jeżeli nawet do takiego doszło właściciel powinien rozwiązywać tą kwestię, wśród swoich pracowników odpowiedzialnych za to

We keep only the reviews in french

In [37]:
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
print(stopwords.fileids())

['arabic', 'azerbaijani', 'basque', 'bengali', 'catalan', 'chinese', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hebrew', 'hinglish', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


In [38]:
def detecte_langage(message):
    # on definie un dictionnaire vide
    # {langue : nbre de stopwords communs 
    # entre langue et les mots de message}
    languages_shared_words = {}
    # tokenization en mots
    words = wordpunct_tokenize(message)
    for language in stopwords.fileids():
        # stopwords pour chaque langue
        stopwords_liste = stopwords.words(language)
        # on retire les doublons
        words = set(words)
        # les mots communs entre stopwords 
        # d'une langue et les mots de message
        common_elements = words.intersection(stopwords_liste)
        # ajout du couple au dictionnaire
        languages_shared_words[language] = len(common_elements)
    # on retourne la langue avec le max de mots commun
    return  max(languages_shared_words, key = languages_shared_words.get)

In [39]:
# on applique la fonction pour chaque message sur la colonne review et on ne garde que les lignes ou la langue est en francais
revsample_df["langue"] = revsample_df["review"].apply(detecte_langage)

In [40]:
revsample_df[revsample_df["langue"] == "french"].count()

company_name    7729
score           7729
review_score    7729
review_title    7729
review_text     7729
category        7729
review          7729
langue          7729
dtype: int64

In [41]:
revsample_df_fr = revsample_df[revsample_df["langue"] == "french"]

In [58]:
revsample_df_fr.to_csv("data/revsample_df_fr.csv", sep=";")

In [5]:
revsample_df_fr = pd.read_csv("data/revsample_df_fr.csv", sep=";")

### Trying CamemBERT model : 

In [6]:
from transformers import CamembertTokenizer

c:\Users\gdetr\Documents\ESILV A5\Machine Learning for NLP\NLP_Project\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# let's use Cambembert model for French
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")

sentencepiece.bpe.model: 100%|██████████| 811k/811k [00:00<00:00, 2.50MB/s]
c:\Users\gdetr\Documents\ESILV A5\Machine Learning for NLP\NLP_Project\myenv\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gdetr\.cache\huggingface\hub\models--camembert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message

In [ ]:
# # we tokenize the reviews
# revsample_df_fr["review"] = revsample_df_fr["review"].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

# revsample_df_fr["review"].values[0]

In [8]:
# we split the dataset into train and test
from sklearn.model_selection import train_test_split

train, test = train_test_split(revsample_df_fr, test_size=0.2, random_state=42)

train.shape, test.shape

((6183, 9), (1546, 9))

In [9]:
train.head(3)

,Unnamed: 0,company_name,score,review_score,review_title,review_text,category,review,langue
1512,1638,Plaquedeces.fr,"4,6",5,Plaque réalisé très vite et très bien livraiso...,Plaque commandée un vendredi tard dans la soir...,construction_manufactoring,Plaque realise tres vite et tres bien livraiso...,french
2404,2598,Recharge.fr,"4,3",5,Site fiable,J'ai fait plusieurs rechargement en passant pa...,events_entertainment,Site fiable J'ai fait plusieurs rechargement e...,french
5167,5612,Taskrabbit France,"4,5",5,Service impeccable,J'ai acheté un meuble chez Ikea et j'ai demand...,home_services,Service impeccable J'ai achete un meuble chez ...,french


In [10]:
# function to encode the reviews with a max_length of 512
max_length = 512
def encode_reviews(tokenizer, reviews, max_length):
    token_ids = np.zeros(shape=(len(reviews), max_length),
                         dtype=np.int32)
    for i, review in enumerate(reviews):
        encoded = tokenizer.encode(review, max_length=max_length)
        token_ids[i, 0:len(encoded)] = encoded
    attention_mask = (token_ids != 0).astype(np.int32)
    return {"input_ids": token_ids, "attention_mask": attention_mask}

In [11]:
# we encode the reviews
train_encoded = encode_reviews(tokenizer, train["review"].values, max_length)
test_encoded = encode_reviews(tokenizer, test["review"].values, max_length)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [12]:
train_encoded

{'input_ids': array([[    5, 21921,   343, ...,     0,     0,     0],
        [    5,  1082,  5758, ...,     0,     0,     0],
        [    5,  2953, 10931, ...,     0,     0,     0],
        ...,
        [    5,    74,  4324, ...,     0,     0,     0],
        [    5,  9519,    22, ...,     0,     0,     0],
        [    5,  1082, 14951, ...,     0,     0,     0]]),
 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])}

In [13]:
# creation of train and test labels
train_labels = train["review_score"].values
test_labels = test["review_score"].values

train_labels

array([5, 5, 5, ..., 5, 5, 5], dtype=int64)

In [14]:
import tensorflow as tf

In [15]:
from transformers import TFCamembertForSequenceClassification

In [16]:
# now, we train the Camembert model
# loading the Camembert model
camembert_model = TFCamembertForSequenceClassification.from_pretrained("camembert-base")

# defining the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-6, epsilon=1e-08)

# defining the loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) # 'sparse_categorical_crossentropy' for integers

model.safetensors: 100%|██████████| 445M/445M [00:03<00:00, 118MB/s] 


All PyTorch model weights were used when initializing TFCamembertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFCamembertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
#we verify the range of the labels (between 0 and 5) in the train and test datasets
print(np.unique(train_labels))
print(np.unique(test_labels))

[1 2 3 4 5]
[1 2 3 4 5]


In [18]:
from sklearn.preprocessing import normalize

In [19]:
# normalization of the values of the array train_labels and test_labels
train_labels = normalize(train_labels.reshape(1, -1), norm="max").reshape(-1)
test_labels = normalize(test_labels.reshape(1, -1), norm="max").reshape(-1)

In [20]:
print(np.unique(train_labels))
print(np.unique(test_labels))

[0.2 0.4 0.6 0.8 1. ]
[0.2 0.4 0.6 0.8 1. ]


In [21]:
train_labels.dtype

dtype('float64')

In [ ]:
# we compile the model
camembert_model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"]) 

initial_weights = camembert_model.get_weights()
camembert_model.set_weights(initial_weights)

In [ ]:
# we train the model
camembert_model.fit(train_encoded, train_labels, epochs=3, batch_size=16)

Epoch 1/3


In [ ]:
# # we evaluate the model
# camembert_model.evaluate(test_encoded, test_labels)

# # we predict the test labels
# test_pred = camembert_model.predict(test_encoded)

# test_pred

### With RNN-LSTM model

In [22]:
### With RNN-LSTM model
import keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [23]:
# we tokenize the reviews
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train["review"].values)

train_sequences = tokenizer.texts_to_sequences(train["review"].values)
test_sequences = tokenizer.texts_to_sequences(test["review"].values)

train_sequences[0]

[493,
 407,
 9,
 256,
 3,
 9,
 26,
 31,
 44,
 3,
 56,
 2258,
 493,
 1243,
 10,
 1424,
 507,
 28,
 6,
 1839,
 90,
 5,
 3581,
 8,
 18,
 8380,
 5,
 1190,
 12,
 1467,
 8381,
 8,
 14,
 8382,
 5,
 1701,
 47,
 150,
 732,
 6,
 31,
 927,
 6,
 493,
 16,
 364,
 12,
 9,
 56,
 2258]

In [24]:
# we pad the sequences
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post")
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding="post", truncating="post")

train_padded[0]

array([ 493,  407,    9,  256,    3,    9,   26,   31,   44,    3,   56,
       2258,  493, 1243,   10, 1424,  507,   28,    6, 1839,   90,    5,
       3581,    8,   18, 8380,    5, 1190,   12, 1467, 8381,    8,   14,
       8382,    5, 1701,   47,  150,  732,    6,   31,  927,    6,  493,
         16,  364,   12,    9,   56, 2258,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [25]:
# we create the model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout

model = Sequential()
model.add(Embedding(10000, 128, input_length=max_length))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 512, 128)          1280000   
                                                                 
 bidirectional (Bidirection  (None, 512, 128)          98816     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout_38 (Dropout)        (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 6

In [26]:
# we compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [27]:
# we train the model
model.fit(train_padded, train_labels, epochs=3, batch_size=16)

Epoch 1/3


387/387 [==============================] - 123s 307ms/step - loss: 0.2582 - accuracy: 0.7252
Epoch 2/3
387/387 [==============================] - 114s 295ms/step - loss: 0.1666 - accuracy: 0.7297
Epoch 3/3
387/387 [==============================] - 114s 296ms/step - loss: 0.1564 - accuracy: 0.7299


In [28]:
# we evaluate the model
model.evaluate(test_padded, test_labels)

49/49 [==============================] - 6s 91ms/step - loss: 0.1549 - accuracy: 0.7316


[0.15491537749767303, 0.7315653562545776]

In [29]:
# we predict the test labels
test_pred = model.predict(test_padded)
test_pred

49/49 [==============================] - 6s 111ms/step


array([[0.82217866],
       [0.99999905],
       [0.99998844],
       ...,
       [0.99999905],
       [0.99999857],
       [0.99999785]], dtype=float32)

In [ ]:
### With RNN-LSTM model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# we tokenize the reviews
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train["review"].values)

train_sequences = tokenizer.texts_to_sequences(train["review"].values)
test_sequences = tokenizer.texts_to_sequences(test["review"].values)

train_sequences[0]

# we pad the sequences
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post")
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding="post", truncating="post")

train_padded[0]

# we create the model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout

model = Sequential()
model.add(Embedding(10000, 128, input_length=max_length))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

model.summary()

# we compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# we train the model
model.fit(train_padded, train_labels, epochs=3, batch_size=16)

# we evaluate the model
model.evaluate(test_padded, test_labels)

# we predict the test labels
test_pred = model.predict(test_padded)

test_pred

# we save the model
model.save("model.h5")